# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from api_keys import g_key
gmaps.configure(api_key=g_key)
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("weather data.csv")
weather_data.dropna(inplace=True)
weather_data.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tiksi,71.6872,128.8694,12.17,59,100,4.94,RU,1660179645
1,mar del plata,-38.0023,-57.5575,10.05,86,0,10.36,AR,1660179525
2,punta arenas,-53.1500,-70.9167,5.06,80,100,4.94,CL,1660179647
3,qaanaaq,77.4840,-69.3632,1.71,65,30,4.72,GL,1660179647
4,hithadhoo,-0.6000,73.0833,27.75,73,1,9.86,MV,1660179648
5,tautira,-17.7333,-149.1500,25.08,72,100,15.14,PF,1660179649
6,alofi,-19.0595,-169.9187,27.94,65,40,9.22,NU,1660179573
7,bredasdorp,-34.5322,20.0403,12.10,92,100,2.39,ZA,1660179438
8,new norfolk,-42.7826,147.0587,12.18,73,97,2.84,AU,1660179650
9,upernavik,72.7868,-56.1549,4.90,87,100,7.29,GL,1660179652


In [3]:
weather_data.count()

City          564
Lat           564
Lng           564
Max Temp      564
Humidity      564
Cloudiness    564
Wind Speed    564
Country       564
Date          564
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
city_locations = weather_data[["Lat", "Lng"]]

In [5]:
humidity = weather_data["Humidity"]

In [6]:
fig = gmaps.figure()

In [7]:
heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, dissipating=False, max_intensity = 10, point_radius = 1)

In [8]:
fig.add_layer(heat_layer)

In [10]:
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
hotel_df = weather_data[(weather_data["Max Temp"]<27) & (weather_data["Max Temp"]>21) & (weather_data["Cloudiness"]==0)  & (weather_data["Wind Speed"]<4.5)]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [13]:
hotel_df["Hotel Name"] = ""

C:\Users\nickj\anaconda3\anacondatake2\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
#check df to see if column was created
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
42,platteville,42.7342,-90.4785,25.05,78,0,3.44,US,1660179680,
117,zhangjiakou,40.8100,114.8794,23.87,60,0,2.77,CN,1660179622,
220,baherden,38.4362,57.4316,25.64,25,0,4.00,TM,1660179826,
276,arcachon,44.5875,-1.0519,26.18,38,0,3.44,FR,1660179874,
327,bom jesus,-9.0744,-44.3586,21.28,28,0,1.63,BR,1660179915,
379,elbistan,38.2059,37.1983,21.62,45,0,1.92,TR,1660179961,
386,misratah,32.3754,15.0925,26.90,68,0,0.60,LY,1660179967,
396,evansville,37.9748,-87.5558,25.01,83,0,1.01,US,1660179841,
446,gat,31.6100,34.7642,25.38,100,0,0.43,IL,1660179721,
549,kerman,30.2832,57.0788,25.79,25,0,4.16,IR,1660180104,


In [15]:
params = {"key":g_key,
          "radius": 5000
    
}

In [16]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lon = row["Lng"]
    params["location"] = f"{lat}, {lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_name = requests.get(base_url, params=params)
    hotel_name = hotel_name.json()
    try:
        #use index one as 0 contains information on the city, not the hotel name
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][1]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "None nearby"


C:\Users\nickj\anaconda3\anacondatake2\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [17]:
hotel_name["results"][1]["name"]

'Melli Bank'

In [18]:
#check to see the type it is stored in
type(hotel_name)

dict

In [19]:
#it is a dictionary, so check to see the keys
hotel_name.keys()

dict_keys(['html_attributions', 'next_page_token', 'results', 'status'])

In [20]:
hotel_name["results"][1]["name"]

'Melli Bank'

In [21]:
#check to see how many hotels there are
len(hotel_name)

4

In [22]:
#use index 0 to get the first result
hotel_name["results"]

[{'geometry': {'location': {'lat': 30.2839379, 'lng': 57.0833628},
   'viewport': {'northeast': {'lat': 30.34591701163794,
     'lng': 57.19688428199603},
    'southwest': {'lat': 30.20018490395793, 'lng': 56.93561554334563}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  'icon_background_color': '#7B9EB0',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
  'name': 'Kerman',
  'photos': [{'height': 501,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/112744456641825994638">Dr. Mahdi HosseiniB</a>'],
    'photo_reference': 'AeJbb3dpMI4G1S_SDkpLHwA2wDVHBlUVua6GVguoWbhaUdisIYfDW9uJfodPPhO17WjCUYELRYaTmjnsT-YliN-r5f7C6uhgPjcnYnfflnGnCdUq5aDuBHb7eoxsw5in7jetTLVgrZHUjCikG6FWC_TW5qjEb2QlmTtwzXwjDTru2r4emcnP',
    'width': 850}],
  'place_id': 'ChIJ0dCx21EYAj8Rto7X9VV0H80',
  'reference': 'ChIJ0dCx21EYAj8Rto7X9VV0H80',
  'scope': 'GOOGLE',
  'types': ['locality', 'political'],
  

In [23]:
hotel_df.head(20)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
42,platteville,42.7342,-90.4785,25.05,78,0,3.44,US,1660179680,University of Wisconsin-Platteville
117,zhangjiakou,40.8100,114.8794,23.87,60,0,2.77,CN,1660179622,Liushutun
220,baherden,38.4362,57.4316,25.64,25,0,4.00,TM,1660179826,TM CELL
276,arcachon,44.5875,-1.0519,26.18,38,0,3.44,FR,1660179874,Hôtel All Suites Le Teich
327,bom jesus,-9.0744,-44.3586,21.28,28,0,1.63,BR,1660179915,Roman Catholic Diocese of Bom Jesus do Gurguéia
379,elbistan,38.2059,37.1983,21.62,45,0,1.92,TR,1660179961,Sword Hotel
386,misratah,32.3754,15.0925,26.90,68,0,0.60,LY,1660179967,Faculty of Arts - Misurata University
396,evansville,37.9748,-87.5558,25.01,83,0,1.01,US,1660179841,University of Evansville
446,gat,31.6100,34.7642,25.38,100,0,0.43,IL,1660179721,Tel Erani / תל ערני
549,kerman,30.2832,57.0788,25.79,25,0,4.16,IR,1660180104,Melli Bank


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [26]:
# Add marker layer ontop of heat map
# fig = gmaps.figure()
# heat_layer = gmaps.heatmap_layer(locations, weights= humidity, dissipating=False, max_intensity=10, point_radius = 1)
# fig.add_layer(heat_layer)
# Display figure
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))